In [ ]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%pprint
%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
#pandemic_df = s.load_object('pandemic_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])

dir()

Pretty printing has been turned OFF
C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\ipynb\Presidential Popularity.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'nx', 'os', 'pd', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_st

In [3]:

tables_url = 'https://en.wikipedia.org/wiki/Historical_rankings_of_presidents_of_the_United_States'
tables_list = get_page_tables(tables_url)

[(6, (46, 24)), (0, (45, 24)), (4, (45, 24)), (5, (45, 14)), (2, (42, 4)), (3, (11, 8)), (1, (17, 3)), (7, (12, 2))]


In [22]:

cn_dict = {}
cn_dict['Bg'] = 'Background'
cn_dict['PL'] = 'Party_Leadership'
cn_dict['CAb'] = 'Communication_Ability'
cn_dict['RC'] = 'Relations_With_Congress'
cn_dict['CAp'] = 'Court_Appointments'
cn_dict['HE'] = 'Handling_of_Economy'
cn_dict['L'] = 'Luck'
cn_dict['AC'] = 'Ability_to_Compromise'
cn_dict['WR'] = 'Willing_to_Take_Risks'
cn_dict['EAp'] = 'Executive_Appointments'
cn_dict['OA'] = 'Overall_Ability'
cn_dict['Im'] = 'Imagination'
cn_dict['DA'] = 'Domestic_Accomplishments'
cn_dict['Int'] = 'Integrity'
cn_dict['EAb'] = 'Executive_Ability'
cn_dict['FPA'] = 'Foreign_Policy_Accomplishments'
cn_dict['LA'] = 'Leadership_Ability'
cn_dict['IQ'] = 'Intelligence'
cn_dict['AM'] = 'Avoid_Crucial_Mistakes'
cn_dict['EV'] = 'Experts_View'
cn_dict['O'] = 'Overall'
cn_dict['Seq.'] = 'Office_Rank'
cn_dict['President'] = 'President'
cn_dict['Political party'] = 'Political_Party'

In [29]:

sienna_expert_poll2010_df = tables_list[6].copy()
sienna_expert_poll2010_df.columns = [cn_dict[cn].lower() for cn in sienna_expert_poll2010_df.columns]
sienna_expert_poll2010_df = sienna_expert_poll2010_df.iloc[1:-1]
sienna_expert_poll2010_df = sienna_expert_poll2010_df.applymap(lambda x: str(x).split('[')[0])
for cn in sienna_expert_poll2010_df.columns:
    if cn not in ['president', 'political_party']:
        sienna_expert_poll2010_df[cn] = sienna_expert_poll2010_df[cn].map(lambda x: re.sub('[^\d]+', '', str(x)))
        sienna_expert_poll2010_df[cn] = pd.to_numeric(sienna_expert_poll2010_df[cn], errors='coerce')
sienna_expert_poll2010_df.set_index('president', inplace=True)
sienna_expert_poll2010_df.index.name = 'President'

def f(x):
    if '_' not in x:
        x = f'presidential_{x}'
    
    return x

sienna_expert_poll2010_df.columns = [f(cn) for cn in sienna_expert_poll2010_df.columns]
sienna_expert_poll2010_df.loc['Grover Cleveland', 'office_rank'] = 24
pn_dict = {'William H. Taft': 'William Howard Taft', 'Barack Obama': 'Barack H. Obama'}
sienna_expert_poll2010_df = sienna_expert_poll2010_df.rename(index=pn_dict)
s.store_objects(sienna_expert_poll2010_df=sienna_expert_poll2010_df)
sienna_expert_poll2010_df.sample(5).T

Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\sienna_expert_poll2010_df.pickle


President,James K. Polk,Barack H. Obama,James Buchanan,Warren G. Harding,Thomas Jefferson
office_rank,11,44,15,29,3
political_party,Democratic,Democratic,Democratic,Republican,Democratic-Republican
presidential_background,19,24,36,39,2
presidential_imagination,10,11,43,41,2
presidential_integrity,23,13,40,42,14
presidential_intelligence,23,9,39,43,1
presidential_luck,9,15,42,33,8
willing_to_take_risks,7,23,41,40,5
ability_to_compromise,18,16,40,34,14
executive_ability,7,18,42,40,6


In [28]:

us_presidents_df = s.load_object('us_presidents_df')
pn_dict = {'William H. Taft': 'William Howard Taft', 'Barack Obama': 'Barack H. Obama'}
us_presidents_df = us_presidents_df.rename(index=pn_dict)
s.store_objects(us_presidents_df=us_presidents_df)
us_presidents_df.sample(5).T

Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\us_presidents_df.pickle


President,William Henry Harrison,Franklin D. Roosevelt,William Howard Taft,Franklin Pierce,Chester A. Arthur
date_of_birth,1773-02-09 00:00:00,1882-01-30 00:00:00,1857-09-15 00:00:00,1804-11-23 00:00:00,1829-10-05 00:00:00
birthplace,Charles City County,Hyde Park,Cincinnati,Hillsborough,Fairfield
state_of_birth,Virginia,New York,Ohio,New Hampshire,Vermont
office_rank,9,32,27,14,21
reign_begin,1841-03-04 00:00:00,1933-03-04 00:00:00,1909-03-04 00:00:00,1853-03-04 00:00:00,1881-09-19 00:00:00
reign_end,1841-04-04 00:00:00,1945-04-12 00:00:00,1913-03-04 00:00:00,1857-03-04 00:00:00,1885-03-04 00:00:00
generation_name,Compromise,Missionary,Progressive,Transcendental,Gilded
turning_name,The Transcendental Awakening,The Great Depression & World War II,World War I & Prohibition,The Mexican War & Sectionalism,Reconstruction & Gilded Age
turning_number,2,4,3,3,1


In [30]:

us_presidents_df = s.load_object('us_presidents_df')
set(us_presidents_df.index).symmetric_difference(set(sienna_expert_poll2010_df.index))

set()

In [34]:

us_presidents_df = us_presidents_df.merge(sienna_expert_poll2010_df, left_index=True, right_index=True)
[f'us_presidents_df.{fn}' for fn in dir(us_presidents_df) if 'dup' in fn.lower()]

['us_presidents_df.drop_duplicates', 'us_presidents_df.duplicated']

In [38]:

us_presidents_df.duplicated(subset=['office_rank_x', 'office_rank_y']).any()

False

In [43]:

us_presidents_df = us_presidents_df.drop(columns=['office_rank_y']).rename(columns={'office_rank_x': 'office_rank'})

In [45]:

s.store_objects(us_presidents_df=us_presidents_df)
us_presidents_df.sample(5).T

Pickling to C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\us_presidents_df.pickle


President,James K. Polk,Jimmy Carter,Andrew Johnson,Grover Cleveland,James A. Garfield
date_of_birth,1795-11-02 00:00:00,1924-10-01 00:00:00,1808-12-29 00:00:00,1837-03-18 00:00:00,1831-11-19 00:00:00
birthplace,Pineville,Plains,Raleigh,Caldwell,Moreland Hills
state_of_birth,North Carolina,Georgia,North Carolina,New Jersey,Ohio
office_rank,11,39,17,24,20
reign_begin,1845-03-04 00:00:00,1977-01-20 00:00:00,1865-04-15 00:00:00,1893-03-04 00:00:00,1881-03-04 00:00:00
reign_end,1849-03-04 00:00:00,1981-01-20 00:00:00,1869-03-04 00:00:00,1897-03-04 00:00:00,1881-09-19 00:00:00
generation_name,Transcendental,G.I.,Transcendental,Gilded,Gilded
turning_name,The Mexican War & Sectionalism,The Consciousness Revolution,Reconstruction & Gilded Age,The Third Great Awakening,Reconstruction & Gilded Age
turning_number,3,2,1,2,1
political_party,Democratic,Democratic,Democratic,Democratic,Republican


In [46]:

[f'us_presidents_df.{fn}' for fn in dir(us_presidents_df) if 'cor' in fn.lower()]

['us_presidents_df.corr', 'us_presidents_df.corrwith', 'us_presidents_df.from_records', 'us_presidents_df.to_records']

In [70]:

generations_df = s.load_object('generations_df')
for cn in ['birth_year_begin', 'birth_year_end', 'turning_year_begin', 'turning_year_end']:
    cn_dict = generations_df[cn].to_dict()
    us_presidents_df[cn] = us_presidents_df.generation_name.map(lambda x: cn_dict[x])

In [71]:

us_presidents_df.columns.tolist()

['date_of_birth', 'birthplace', 'state_of_birth', 'office_rank', 'reign_begin', 'reign_end', 'generation_name', 'turning_name', 'turning_number', 'political_party', 'presidential_background', 'presidential_imagination', 'presidential_integrity', 'presidential_intelligence', 'presidential_luck', 'willing_to_take_risks', 'ability_to_compromise', 'executive_ability', 'leadership_ability', 'communication_ability', 'overall_ability', 'party_leadership', 'relations_with_congress', 'court_appointments', 'handling_of_economy', 'executive_appointments', 'domestic_accomplishments', 'foreign_policy_accomplishments', 'avoid_crucial_mistakes', 'experts_view', 'presidential_overall', 'birth_year_begin', 'birth_year_end', 'turning_year_begin', 'turning_year_end']

In [79]:

import numpy as np

max_corr = 0
max_row = np.nan
max_col = np.nan
us_presidents_df['office_rank'] = pd.to_numeric(us_presidents_df['office_rank'], errors='coerce')
corr_df = us_presidents_df.corr()
old_columns = ['office_rank', 'turning_number', 'birth_year_begin', 'birth_year_end', 'turning_year_begin',
               'turning_year_end']
new_columns = ['presidential_background', 'presidential_imagination', 'presidential_integrity',
               'presidential_intelligence', 'presidential_luck', 'willing_to_take_risks', 'ability_to_compromise',
               'executive_ability', 'leadership_ability', 'communication_ability', 'overall_ability', 'party_leadership',
               'relations_with_congress', 'court_appointments', 'handling_of_economy', 'executive_appointments',
               'domestic_accomplishments', 'foreign_policy_accomplishments', 'avoid_crucial_mistakes', 'experts_view',
               'presidential_overall']
mask_series = corr_df.index.isin(new_columns)
for row_index, row_series in corr_df[mask_series].iterrows():
    for col_index, col_value in row_series.iteritems():
        if (col_index in old_columns) and (col_value < 1.0) and (col_value > max_corr):
            max_corr = col_value
            max_row = row_index
            max_col = col_index
print("us_presidents_df.plot.scatter(x='{}', y='{}');".format(max_row, max_col))

us_presidents_df.plot.scatter(x='presidential_integrity', y='birth_year_end');


In [87]:

import numpy as np

max_corr = 0
max_row = np.nan
max_col = np.nan
us_presidents_df['office_rank'] = pd.to_numeric(us_presidents_df['office_rank'], errors='coerce')
corr_df = us_presidents_df.corr()
old_columns = ['presidential_integrity']
new_columns = ['presidential_background', 'presidential_imagination',
               'presidential_intelligence', 'presidential_luck', 'willing_to_take_risks', 'ability_to_compromise',
               'executive_ability', 'leadership_ability', 'communication_ability', 'overall_ability', 'party_leadership',
               'relations_with_congress', 'court_appointments', 'handling_of_economy', 'executive_appointments',
               'domestic_accomplishments', 'foreign_policy_accomplishments', 'avoid_crucial_mistakes', 'experts_view',
               'presidential_overall']
mask_series = corr_df.index.isin(new_columns)
for row_index, row_series in corr_df[mask_series].iterrows():
    for col_index, col_value in row_series.iteritems():
        if (col_index in old_columns) and (col_value < 1.0) and (col_value > max_corr):
            max_corr = col_value
            max_row = row_index
            max_col = col_index
print("us_presidents_df.plot.scatter(x='{}', y='{}');".format(max_row, max_col))

us_presidents_df.plot.scatter(x='avoid_crucial_mistakes', y='presidential_integrity');


In [110]:

%matplotlib inline
save_fig_as_various(us_presidents_df, 'Wile E. Coyote Syndrone: Avoiding Crucial Mistakes by having Integrity')

'DataFrame' object has no attribute 'savefig'
'DataFrame' object has no attribute 'savefig'
'DataFrame' object has no attribute 'savefig'


In [107]:

[fn for fn in dir() if 'save' in fn.lower()]

['save_fig_as_various']

In [90]:

columns_list = ['office_rank', 'generation_name', 'turning_name', 'turning_number', 'political_party',
                'presidential_integrity', 'avoid_crucial_mistakes', 'experts_view', 'presidential_overall',
                'birth_year_begin', 'birth_year_end', 'turning_year_begin', 'turning_year_end']
us_presidents_df[columns_list].sort_values('presidential_integrity').tail(5).T

President,James Buchanan,Andrew Johnson,Warren G. Harding,Richard Nixon,Donald Trump
office_rank,15,17,29,37,45
generation_name,Compromise,Transcendental,Missionary,G.I.,Baby Boom
turning_name,The Mexican War & Sectionalism,Reconstruction & Gilded Age,World War I & Prohibition,The Consciousness Revolution,The Global Financial Crisis
turning_number,3,1,3,2,4
political_party,Democratic,Democratic,Republican,Republican,Republican
presidential_integrity,40,41,42,43,44
avoid_crucial_mistakes,44,43,39,42,41
experts_view,44,43,41,38,42
presidential_overall,43,44,41,29,42
birth_year_begin,1767,1792,1860,1901,1943


In [111]:

columns_list = ['office_rank', 'generation_name', 'turning_name', 'turning_number', 'political_party',
                'presidential_integrity', 'avoid_crucial_mistakes', 'experts_view', 'presidential_overall',
                'birth_year_begin', 'birth_year_end', 'turning_year_begin', 'turning_year_end']
us_presidents_df[columns_list].sort_values('avoid_crucial_mistakes').tail(5).T

President,Herbert Hoover,Donald Trump,Richard Nixon,Andrew Johnson,James Buchanan
office_rank,31,45,37,17,15
generation_name,Missionary,Baby Boom,G.I.,Transcendental,Compromise
turning_name,The Great Depression & World War II,The Global Financial Crisis,The Consciousness Revolution,Reconstruction & Gilded Age,The Mexican War & Sectionalism
turning_number,4,4,2,1,3
political_party,Republican,Republican,Republican,Democratic,Democratic
presidential_integrity,15,44,43,41,40
avoid_crucial_mistakes,40,41,42,43,44
experts_view,35,42,38,43,44
presidential_overall,36,42,29,44,43
birth_year_begin,1860,1943,1901,1792,1767


In [78]:

get_column_descriptions(us_presidents_df)

,column_name,dtype,count_blanks,count_uniques,count_zeroes,has_dates,min_value,max_value,only_integers
0,date_of_birth,datetime64[ns],0,44,0,True,1732-02-22 00:00:00,1961-08-04 00:00:00,NaN
1,reign_begin,datetime64[ns],0,45,0,True,1789-04-30 00:00:00,2017-01-20 00:00:00,NaN
2,reign_end,datetime64[ns],1,45,0,True,1797-03-04 00:00:00,2017-01-20 00:00:00,NaN
3,turning_number,int64,0,4,0,True,1,4,True
4,presidential_background,int64,0,43,0,True,1,43,True
5,presidential_imagination,int64,0,43,0,True,1,43,True
6,presidential_integrity,int64,0,44,0,True,1,44,True
7,presidential_intelligence,int64,0,44,0,True,1,44,True
8,presidential_luck,int64,0,44,0,True,1,44,True
9,willing_to_take_risks,int64,0,41,0,True,1,41,True
